In [1]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np

ModuleNotFoundError: No module named 'torch'

In [3]:
df = pd.read_csv('spi_results.csv')
# features = ['tmax_m', 'tmax_max', 'tmax_min', 'tmin_m', 'tmin_min', 'tmin_max', 'ntmin_0', 'rrr24', 'sshn', 'tm_m', 't_03_m', 't_09_m', 't_15_m']
features = ['tmax_m', 'tmin_m', 'rrr24', 'SPI']

target = 'SPI'

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[features].values)

# scaled_features = scaler.fit_transform(df[features])
# scaled_target = scaler.fit_transform(df[[target]])

# data = torch.tensor(scaled_features, dtype=torch.float32)
# target = torch.tensor(scaled_target, dtype=torch.float32)

# Create sliding windows for LSTM
def create_sequences(data, window_size):
    sequences = []
    targets = []
    for i in range(len(data) - window_size):
        seq = data[i:i+window_size]
        target = data[i+window_size, -1]  # target is the last column (SPI)
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

window_size = 12  # 12 months = 1 year
X, y = create_sequences(scaled_data, window_size)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)


# Hyperparameters
input_size = len(features)
hidden_size = 64
num_layers = 2
output_size = 1
seq_length = 10
batch_size = 32
num_epochs = 100
learning_rate = 0.001




device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_tensor = X_tensor.to(device)
y_tensor = y_tensor.to(device)
print(f"Using {device} device")

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    

dataset = TimeSeriesDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size, shuffle=True)
# Initialize the model, loss function, and optimizer
model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
print(model)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Using cuda device
LSTMModel(
  (lstm): LSTM(4, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)


In [4]:
# Training loop
for epoch in range(num_epochs):
    for inputs, labels in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the model
torch.save(model.state_dict(), 'lstm_model_vahid.pth')
# Load the saved model for inference



Epoch [10/100], Loss: 0.0723
Epoch [20/100], Loss: 0.0746
Epoch [30/100], Loss: 0.0792
Epoch [40/100], Loss: 0.0523
Epoch [50/100], Loss: 0.0570
Epoch [60/100], Loss: 0.0523
Epoch [70/100], Loss: 0.0456
Epoch [80/100], Loss: 0.0471
Epoch [90/100], Loss: 0.0375
Epoch [100/100], Loss: 0.0389


In [5]:
model.load_state_dict(torch.load('lstm_model_vahid.pth'))
model.eval()

C:\Users\varas\AppData\Local\Temp\ipykernel_17284\3253695867.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('lstm_model_vahid.pth'))


LSTMModel(
  (lstm): LSTM(4, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

In [6]:
# Prepare test data for prediction
test_data = torch.tensor(scaled_data[-seq_length:], dtype=torch.float32).unsqueeze(0)
test_tensor = test_data.to(device)

# Make predictions
with torch.no_grad():
    predictions = model(test_tensor)
predictions_np = predictions.cpu().numpy()
dummy_array = np.zeros((predictions_np.shape[0], scaler.min_.shape[0]))
dummy_array[:, -1] = predictions_np[:, 0] 
# Inverse transform predictions to original scale
# predictions_rescaled = scaler.inverse_transform(predictions.numpy())
predictions_rescaled = scaler.inverse_transform(dummy_array)
predicted_spi = predictions_rescaled[:, -1]

# Print the predictions
print("Predicted SPI:", predicted_spi)

Predicted SPI: [-0.06656761]
